<a href="https://colab.research.google.com/github/Isiumlord/GoogleCloudPlatformStudy/blob/main/JobPipeline_v0_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TAREFAS

* Ignorar colunas não utilizáveis (deletar colunas) ✔
* Ajustar campos de data (mudar disposição) ✔
* Concatenações (juntar campos diversos: pode ser por junção simples de string, cálculo entre colunas ou cruzamento de dados) ✔
* Domínio (padrão do cliente) - fazer "De x Para" com o padrão de campos da Dasa - Exemplo: Feminino para F) ✔

DATASET TRABALHADO
* Premier League Match Data (https://www.kaggle.com/datasets/evangower/premier-league-match-data)

Instalando Apache Beam

In [ ]:
!pip install apache-beam

Importando Bibliotecas

In [39]:
import apache_beam as beam
from apache_beam.io import ReadFromText
import pandas as pd

Declarando Variavel Global

In [40]:
pipeline = beam.Pipeline()

Criando Funções de Transformações em Pandas

* Lendo o Arquivo .csv
* Transformando-o em Dataframe
* Transformando Coluna `Date` em `datetime`
* Formatando a data para `dia/mês/ano`

In [41]:
def formatandoData(df):

    # Lendo Arquivo
    df = pd.read_csv("/content/futebol.csv", sep=",")
    
    # Formatando data para `dia/mês/ano`
    df['Date'] = pd.to_datetime(df.Date)
    df['Date'] = df['Date'].dt.strftime('%d/%m/%Y')

    # Formatando data para `mês/dia/ano`
    #df['Date'] = pd.to_datetime(df.Date)
    #df['Date'] = df['Date'].dt.strftime('%m/%d/%Y')

    return df

Excluindo Colunas: FTHG, FTAG, HTHG, HTAG, HTR, HST, AST, HF, AF, HC, AC, HY, AY, HR, AR

In [42]:
def excluirColunas(df):
    df.drop(["FTHG",
             "FTAG",
             "HTHG",
             "HTAG",
             "HTR", "HST", "AST", "HF", "AF", "HC", "AC", "HY", "AY", "HR", "AR"], axis=1, inplace=True)

    return df

Editando Dados da Coluna `FTR`

In [43]:
def editandoDados(df):
    df.loc[df.FTR == "H", ["FTR"]] = "H-Editado"
    df.loc[df.FTR == "A", ["FTR"]] = "A-Editado"
    df.loc[df.FTR == "D", ["FTR"]] = "D-Editado"

    return df

Concatenando as colunas `HomeTeam` e `AwayTeam` em uma só coluna chamada `Combate`

In [44]:
def concatenando(df):
    df['Combate'] = df['HomeTeam'].map(str) + ' VS ' + df['AwayTeam'].map(str)
    return df

Transformando DataFrame em Dicionário (necessário para rodar no Job Pipeline)

In [45]:
def dicionario(df):
    df_dicionario = df.to_dict('records')
    return df_dicionario

Job Pipeline em Apache Beam

In [ ]:
job = (
    pipeline
    |'Lendo Arquvio' >> beam.io.ReadFromText("/content/futebol.csv", skip_header_lines=0)
    |'Formatando Data' >> beam.Map(formatandoData)
    |'Excluindo Colunas' >> beam.Map(excluirColunas)
    |'Editando Dados' >> beam.Map(editandoDados)
    |'Concatenando Colunas' >> beam.Map(concatenando)
    |'Transformando Dataframe em Dicionario' >> beam.Map(dicionario)
    |'Salvando Resultados em txt' >> beam.io.WriteToText("/content/resultados.txt")
    |"Mostrar resultados" >> beam.Map(print)
)

pipeline.run()